In [6]:
import pygame
from keras.models import load_model
import cv2
import numpy as np
model = load_model('model-005.model')
source=cv2.VideoCapture(0)

# labels_dict={0:'non-punch',1:'punch'}
pygame.init()

win = pygame.display.set_mode((500,500))
pygame.display.set_caption("First Game")


bg = pygame.image.load('background_1.png')
char = pygame.image.load('standing_1_2.png')
punching=[pygame.image.load('0_1.png'),pygame.image.load('1_1.png'),pygame.image.load('2_1.png'),pygame.image.load('3_1.png'),pygame.image.load('4_1.png'),pygame.image.load('5_1.png'),pygame.image.load('6_1.png'),pygame.image.load('7_1.png'),pygame.image.load('8_1.png')]
char2= pygame.image.load('standing_2_1.png')
player2dead=[pygame.image.load('1_2.png'),pygame.image.load('2_2.png'),pygame.image.load('3_2.png'),pygame.image.load('4_2.png'),pygame.image.load('5_2.png')]
x = 200
y = 390
x_2=250
y_2=400


#global health_1
#global health_2
health_1=10
health_2=10

clock = pygame.time.Clock()


isPunch=False


hitCount = 0
player_dead_count=0
def healthbar():
    pygame.draw.rect(win, (255,0,0), (380, 40, 100, 30))
    
    pygame.draw.rect(win, (0,255,0), (380, 40, (100-(10*(10-health_2))), 30))
    pygame.draw.rect(win, (255,0,0), (20, 40, 100, 30))
    pygame.draw.rect(win, (0,255,0), (20, 40, 100-(10*(10-health_1)), 30))
def redrawGameWindow(isPunch):
    global hitCount
    global health_2
    global health_1
    global player_dead_count
    win.blit(bg, (0,0))  
    if hitCount + 1 >= 9:
        hitCount = 0
        return
        
    if isPunch:
       
                
        win.blit(punching[hitCount//1], (x,y))
        hitCount += 1
        if health_2!=0:
            
            win.blit(player2dead[0],(x_2,y_2))
    
    else:
        win.blit(char, (x, y))
        hitCount = 0
    if health_2==0:
        hitCount=1
        if(player_dead_count==4):
            win.blit(player2dead[player_dead_count],(x_2,y_2+45))
           
        else:
            win.blit(player2dead[player_dead_count],(x_2,y_2))
        player_dead_count+=1
    elif health_2!=0 and isPunch==False:    
        win.blit(char2,(x_2,y_2))
    healthbar()
    pygame.display.update() 
    


run = True
while(run):

    ret,img=source.read()
    resized=cv2.resize(img,(224,224))
    normalized=resized/255.0
    reshaped=np.reshape(normalized,(1,224,224,3))
    result=model.predict(reshaped)
    label=np.argmax(result,axis=1)[0]
    #cv2.putText(img, labels_dict[label], (20, 20),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break


    clock.tick(9)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False

    keys = pygame.key.get_pressed()
        
    if hitCount ==0:
        if keys[pygame.K_SPACE] or label==1:
        #if label==1:
                isPunch = True
                health_2-=1
                
                
        else:
            isPunch=False
            hitCount=0

    redrawGameWindow(isPunch)
    if player_dead_count==5:
        break
    
cv2.destroyAllWindows()
source.release()
pygame.time.wait(3000)
pygame.quit()